In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

import os, glob, json, pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as sl

import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const

import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
import enterprise_extensions
from enterprise_extensions import models, sampler, model_utils

In [2]:
# import the par and tim files
datadir = './fakes_gwb3'

parfiles = sorted(glob.glob(datadir + '/par/' + '*.par'))
timfiles = sorted(glob.glob(datadir + '/tim/' + '*.tim'))

psrs = []
for p, t in zip(parfiles, timfiles):
    psrname = parfiles[0].split('/')[-1].split('_')[0]
    psr = Pulsar(p, t, ephem='DE438')
    psrs.append(psr)

In [5]:
for psr in psrs:
    print(psr.name)

B1855+09
B1937+21
J0030+0451
J0613-0200
J1012+5307
J1024-0719
J1455-3330
J1600-3053
J1614-2230
J1640+2224
J1643-1224
J1713+0747
J1738+0333
J1741+1351
J1744-1134
J1903+0327
J1909-3744
J1910+1256
J1918-0642
J2010-1323
J2145-0750
J2317+1439


In [7]:
def gwb_ul(psrs_cut):

    # find the maximum time span to set GW frequency sampling
    tmin = [p.toas.min() for p in psrs_cut]
    tmax = [p.toas.max() for p in psrs_cut]
    Tspan = np.max(tmax) - np.min(tmin)
    # define selection by observing backend
    selection = selections.Selection(selections.by_backend)
    # white noise parameters
    # we set these ourselves so we know the most likely values!
    efac = parameter.Constant(1)
    equad = parameter.Constant(0)
    ecorr = parameter.Constant(0)

    # red noise parameters
    log10_A = parameter.LinearExp(-20, -11)
    gamma = parameter.LinearExp(0, 7)

    # GW parameters (initialize with names here to use parameters in common across pulsars)
    log10_A_gw = parameter.LinearExp(-18,-12)('log10_A_gw')
    gamma_gw = parameter.Constant(4.33)('gamma_gw')
    # white noise
    ef = white_signals.MeasurementNoise(efac=efac, selection=selection)
    eq = white_signals.EquadNoise(log10_equad=equad, selection=selection)
    ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection)

    # red noise (powerlaw with 30 frequencies)
    pl = utils.powerlaw(log10_A=log10_A, gamma=gamma)
    rn = gp_signals.FourierBasisGP(spectrum=pl, components=30, Tspan=Tspan)

    # gwb (no spatial correlations)
    cpl = utils.powerlaw(log10_A=log10_A_gw, gamma=gamma_gw)
    gw = gp_signals.FourierBasisGP(spectrum=cpl, components=30, Tspan=Tspan, name='gw')

    # timing model
    tm = gp_signals.TimingModel(use_svd=True) # stabilizing timing model design matrix with SVD
    s = ef + rn + gw + tm

    # intialize PTA
    models = []

    for p in psrs_cut:
        models.append(s(p))

    pta = signal_base.PTA(models)
    outDir = './chains_pta_test'
    sample = sampler.setup_sampler(pta, outdir=outDir)
    x0 = np.hstack([p.sample() for p in pta.params])

    # sampler for N steps
    N = int(3e5)  # normally, we would use 5e6 samples (this will save time)
    sample.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

    chain = np.loadtxt(os.path.join(outDir, 'chain_1.txt'))
    pars = np.loadtxt('./chains_pta_test/pars.txt', dtype=np.unicode_)
    ind = list(pars).index('log10_A_gw')

    UL = model_utils.ul(chain[:,ind])
    return UL

In [17]:
best_list = []
ul_list = []
num_list = []
for i in range(len(psrs)):
    psrscut = []
    # print(i)
    for num in best_list:
        psrscut.append(psrs[num])
        print(psrs[num].name)
    psrscut.append(psrs[i])
    print(psrs[i].name)
    # print(len(psrscut))
    ul_list.append(gwb_ul(psrscut))
    num_list.append(i)
    print(i, ul_list[i])
print(ul_list)

J1713+0747
B1855+09
Adding red noise prior draws...

Finished 3.33 percent in 11.115160 s Acceptance rate = 0.2985Adding DE jump with weight 50
Finished 99.67 percent in 328.669211 s Acceptance rate = 0.383117
Run Complete
0 (3.79568218928607e-15, 3.4555902181947787e-17)
J1713+0747
B1937+21
Adding red noise prior draws...

Finished 3.33 percent in 12.239116 s Acceptance rate = 0.2288Adding DE jump with weight 50
Finished 99.67 percent in 364.580327 s Acceptance rate = 0.367495
Run Complete
1 (3.5780095840097374e-15, 1.876056336233249e-17)
J1713+0747
J0030+0451
Adding red noise prior draws...

Finished 3.33 percent in 10.253848 s Acceptance rate = 0.2993Adding DE jump with weight 50
Finished 99.67 percent in 304.895400 s Acceptance rate = 0.369562
Run Complete
2 (3.7362775019114554e-15, 2.999625849137e-17)
J1713+0747
J0613-0200
Adding red noise prior draws...

Finished 3.33 percent in 11.197261 s Acceptance rate = 0.3394Adding DE jump with weight 50
Finished 99.67 percent in 334.491410 

KeyboardInterrupt: 

In [14]:
with open('ul_data_2psr.txt', 'w') as f:
    np.savetxt(f, ul_list)